In [1]:
import numpy as np
from numba import cfunc
from cminpack_numba import hybrd_sig, hybrd1, hybrd
from scipy.optimize import root

def func(x):
    return [x[0] * np.cos(x[1]) - 4, x[1] * x[0] - x[1] - 5]


@cfunc(hybrd_sig)
def func_numba(udata, n, x, fvec, iflag):
    fvec[0] = x[0] * np.cos(x[1]) - 4
    fvec[1] = x[1] * x[0] - x[1] - 5
    return 0

In [2]:
root(func, [1, 1], method='hybr')

 message: The solution converged.
 success: True
  status: 1
     fun: [ 3.732e-12  1.617e-11]
       x: [ 6.504e+00  9.084e-01]
  method: hybr
    nfev: 17
    fjac: [[-5.625e-01 -8.268e-01]
           [ 8.268e-01 -5.625e-01]]
       r: [-1.091e+00 -1.762e+00 -7.374e+00]
     qtf: [ 6.257e-08  2.401e-08]

In [3]:
%timeit root(func, [1, 1], method='hybr')

28.7 µs ± 442 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [4]:
print(hybrd1(func_numba.address, np.array([1.0, 1.0])))
%timeit hybrd1(func_numba.address, np.array([1.0, 1.0]))

(array([6.50409711, 0.90841421]), array([-7.39657224e-11,  6.40909548e-12]), 1)
3.22 µs ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
print(hybrd(func_numba.address, np.array([1.0, 1.0])))
%timeit hybrd(func_numba.address, np.array([1.0, 1.0]))

(array([6.50409711, 0.90841421]), array([3.73212572e-12, 1.61701763e-11]), array([[-0.56248005, -0.82681085],
       [ 0.82681085, -0.56248005]]), array([-1.0907073 , -1.7621827 , -7.37420598]), array([6.25677405e-08, 2.40104780e-08]), 17, 1)
4.8 µs ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
